In [1]:
import matplotlib.pyplot as plt
import numpy
import time
import numpy as np
import importlib.util as imp
if imp.find_spec("cupy"): #use cupy for GPU support if available
    import cupy
    import cupy as np


import model_io
import data_io
import render
import statsmodels.api as sm


#load a neural network, as well as the MNIST test data and some labels
nn = model_io.read('../models/MNIST/LeNet-5.nn') # 99.23% prediction accuracy

nn.drop_softmax_output_layer() #drop softnax output layer for analyses

X = data_io.read('../data/MNIST/test_images.npy')
Y = data_io.read('../data/MNIST/test_labels.npy')


# transfer pixel values from [0 255] to [-1 1] to satisfy the expected input / training paradigm of the model
# 픽셀 벨류를 0 ~ 255인 애들에서 -1과 1의 값으로 변경
X =  X / 127.5 - 1.

#reshape the vector representations in X to match the requirements of the CNN input
# CNN에 들어갈 수 있도록 Vector를 Reshape(array로)
X = np.reshape(X,[X.shape[0],28,28,1])
# 패딩 진행
X = np.pad(X,((0,0),(2,2),(2,2),(0,0)), 'constant', constant_values = (-1.,))

# transform numeric class labels to vector indicator for uniformity. assume presence of all classes within the label set
# 클래스를 Label화 시키기 위해 Unique 값 지정 및 0으로 초기화
I = Y[:,0].astype(int)
Y = np.zeros([X.shape[0],np.unique(Y).size])
Y[np.arange(Y.shape[0]),I] = 1

loading pickled model from ../models/MNIST/LeNet-5.nn
output layer is not softmax. nothing to do
loading np-formatted data from ../data/MNIST/test_images.npy
loading np-formatted data from ../data/MNIST/test_labels.npy


In [2]:
acc = np.mean(np.argmax(nn.forward(X), axis=1) == np.argmax(Y, axis=1))
# 가장 높은 값의 평균을 가져오고, 예측값 진행(argmax) forward는 pytorch로 코딩한 것.
if not np == numpy: # np=cupy
    acc = np.asnumpy(acc)
print('model test accuracy is: {:0.4f}'.format(acc))

model test accuracy is: 0.9924


In [3]:
I = np.arange(X.shape[0])

In [4]:
for i in I[:10]:
    # i의 순번을 지정하고
    x = X[i:i+1,...]

    #forward pass and prediction
    # 예측하고
    ypred = nn.forward(x)
    print('True Class:     ', np.argmax(Y[i]))
    print('Predicted Class:', np.argmax(ypred),'\n')


True Class:      7
Predicted Class: 7 

True Class:      2
Predicted Class: 2 

True Class:      1
Predicted Class: 1 

True Class:      0
Predicted Class: 0 

True Class:      4
Predicted Class: 4 

True Class:      1
Predicted Class: 1 

True Class:      4
Predicted Class: 4 

True Class:      9
Predicted Class: 9 

True Class:      5
Predicted Class: 5 

True Class:      9
Predicted Class: 9 



In [6]:
nn.get_activations(x)

AttributeError: 'Sequential' object has no attribute 'get_activations'

In [ ]:

    #prepare initial relevance to reflect the model's dominant prediction (ie depopulate non-dominant output neurons)
    # zeros_like를 사용해서 다른 배열과 같은 크기의 배열을 0로 생성하고
    mask = np.zeros_like(ypred)
    # argmax를 사용해서 가장 큰 값 가져오고
    mask[:,np.argmax(ypred)] = 1
    # 초기 Relevance Score 지정하고
    Rinit = ypred*mask


    #compute first layer relevance according to prediction
    #R = nn.lrp(Rinit)                   #as Eq(56) from DOI: 10.1371/journal.pone.0130140
    R = nn.lrp(Rinit,'epsilon',1.)

    R = R.sum(axis=3)
    xs = ((x+1.)/2.).sum(axis=3)

    if not np == numpy: 
        xs = np.asnumpy(xs)
        R = np.asnumpy(R)

    digit = render.digit_to_rgb(xs, scaling = 3)
    hm = render.hm_to_rgb(R, X = xs, scaling = 3, sigma = 2)
    digit_hm = render.save_image([digit,hm],'../heatmap.png')
    data_io.write(R,'../heatmap.npy')
    
    y = R
    a = np.load('../convolution.npy')
    a = np.reshape(a,[a.shape[1]*a.shape[2],1])
    b = np.load('../sumpoll.npy')
    b = np.pad(b,((0,0),(2,2),(2,2),(0,0)))
    b = np.reshape(b,[b.shape[1]*b.shape[2],b.shape[3]])
    c = np.load('../rect.npy')    
    c = np.pad(c,((0,0),(2,2),(2,2),(0,0)))
    c = np.reshape(c,[c.shape[1]*c.shape[2],c.shape[3]])
    
    new_b = np.hstack((b, c))
    new = np.hstack((a, new_b))
    y = np.reshape(y, [y.shape[0]*y.shape[1]*y.shape[2]])
    y_tran = y.transpose()
    new = sm.add_constant(new)
    new = np.nan_to_num(new)
    y = np.nan_to_num(y)
    model = sm.GLSAR(y_tran, new, rho = 4, missing = "raise")
    for i in range(2):
        result = model.fit()
        print("AR coefficients : {0}".format(model.rho))
        rho, sigma = sm.regression.yule_walker(result.resid, order = model.order)
        print("{}step is done".format(i))
        model = sm.GLSAR(y_tran, new, rho)
    find = result.resid
    check = np.reshape(find,[1,32,32])
    
    digit = render.digit_to_rgb(xs, scaling = 3)
    hm_R = render.hm_to_rgb(check, X = xs, scaling = 3, sigma = 2)
    digit_hm = render.save_image([digit,hm, hm_R],'../re_heatmap.png')
    data_io.write(check,'../re_heatmap.npy')
    

    plt.imshow(digit_hm, interpolation = 'none')
    plt.axis('off')
    plt.show()
